In [3]:
#-----------------------------------
# GLOBAL FEATURE EXTRACTION
#-----------------------------------

# organize imports
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mahotas
import glob
import cv2
import os
import h5py

# fixed-sizes for image
fixed_size = tuple((500, 500))

# path to training data
#path = os.getcwd()
train_path = 'dataset/train'
#data_dir_list =os.listdir(train_path)

# no.of.trees for Random Forests
num_trees = 100

# bins for histogram
bins = 8

# train_test_split size
test_size = 0.10

# seed for reproducing same results
seed = 9

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

# get the training labels
train_labels = os.listdir(train_path)

# sort the training labels
train_labels.sort()
print(train_labels)

# empty lists to hold feature vectors and labels
global_features = []
labels = []

i, j = 0, 0
k = 0

# num of images per class
images_per_class = 80

img_data_list = []
# loop over the training data sub-folders
for training_name in train_labels:
    # join the training data path and each species training folder
    img_list=os.listdir(train_path+'/'+training_name)
    print('Loaded the images of dataset-' + '{}\n'.format(training_name))

    # get the current training label
    current_label = training_name

    k = 1
    # loop over the images in each sub-folder

    for x in img_list:
        # get the image file name
        file = train_path +'/'+training_name+'/'+x
        image_path = os.path.join(train_path, training_name, x)
        if os.path.exists(image_path):
            # read the image and resize it to a fixed-size
            image = cv2.imread(file)
            image = cv2.resize(image, fixed_size)
            img_data_list.append(image)
        ####################################
        # Global Feature extraction
        ####################################
        fv_hu_moments = fd_hu_moments(image)
        fv_haralick   = fd_haralick(image)
        fv_histogram  = fd_histogram(image)

        ###################################
        # Concatenate global features
        ###################################
        global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])

        # update the list of labels and feature vectors
        labels.append(current_label)
        global_features.append(global_feature)

        i += 1
        k += 1
    print ("[STATUS] processed folder: {}".format(current_label))
    j += 1

print( "[STATUS] completed Global Feature Extraction...")

# get the overall feature vector size
print ("[STATUS] feature vector size {}".format(np.array(global_features).shape))

# get the overall training label size
print ("[STATUS] training Labels {}".format(np.array(labels).shape))

# encode the target labels
targetNames = np.unique(labels)
le = LabelEncoder()
target = le.fit_transform(labels)
print ("[STATUS] training labels encoded...")

# normalize the feature vector in the range (0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(global_features)
print ("[STATUS] feature vector normalized...")

print ("[STATUS] target labels: {}".format(target))
print ("[STATUS] target labels shape: {}".format(target.shape))

# save the feature vector using HDF5
h5f_data = h5py.File('output/data.h5', 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File('output/labels.h5', 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print ("[STATUS] end of training..")

['bluebell', 'buttercup', 'colts_foot', 'cowslip', 'crocus', 'daffodil', 'daisy', 'dandelion', 'fritillary', 'iris', 'lily_valley', 'pansy', 'snowdrop', 'sunflower', 'tigerlily', 'tulip', 'windflower']
Loaded the images of dataset-bluebell

[STATUS] processed folder: bluebell
Loaded the images of dataset-buttercup

[STATUS] processed folder: buttercup
Loaded the images of dataset-colts_foot

[STATUS] processed folder: colts_foot
Loaded the images of dataset-cowslip

[STATUS] processed folder: cowslip
Loaded the images of dataset-crocus

[STATUS] processed folder: crocus
Loaded the images of dataset-daffodil

[STATUS] processed folder: daffodil
Loaded the images of dataset-daisy

[STATUS] processed folder: daisy
Loaded the images of dataset-dandelion

[STATUS] processed folder: dandelion
Loaded the images of dataset-fritillary

[STATUS] processed folder: fritillary
Loaded the images of dataset-iris

[STATUS] processed folder: iris
Loaded the images of dataset-lily_valley

[STATUS] proce

In [4]:
#-----------------------------------
# TRAINING OUR MODEL
#-----------------------------------

# import the necessary packages
import h5py
import numpy as np
import os
import glob
import cv2
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.externals import joblib

num_trees = 100
test_size = 0.10
seed = 9
fixed_size = tuple((500, 500))
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

# create all the machine learning models
models = []
models.append(('LR', LogisticRegression(random_state=9)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(random_state=9)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=9)))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(random_state=9)))

#fixed_size = tuple((500, 500))
# variables to hold the results and names
results = []
names = []
scoring = "accuracy"

# import the feature vector and trained labels
h5f_data = h5py.File('output/data.h5', 'r')
h5f_label = h5py.File('output/labels.h5', 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()

# verify the shape of the feature vector and labels
print ("[STATUS] features shape: {}".format(global_features.shape))
print ("[STATUS] labels shape: {}".format(global_labels.shape))

print ("[STATUS] training started...")

# split the training and testing data
(trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal) = train_test_split(np.array(global_features),
                                                                                          np.array(global_labels),
                                                                                          test_size= test_size,
                                                                                          random_state= seed)

print ("[STATUS] splitted train and test data...")
print ("Train data  : {}".format(trainDataGlobal.shape))
print ("Test data   : {}".format(testDataGlobal.shape))
print ("Train labels: {}".format(trainLabelsGlobal.shape))
print ("Test labels : {}".format(testLabelsGlobal.shape))

# filter all the warnings
import warnings
warnings.filterwarnings('ignore')

# 10-fold cross validation
for name, model in models:
    kfold = KFold(n_splits=10, random_state=7)
    cv_results = cross_val_score(model, trainDataGlobal, trainLabelsGlobal, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# boxplot algorithm comparison
fig = pyplot.figure()
fig.suptitle('Machine Learning algorithm comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()



[STATUS] features shape: (1190, 532)
[STATUS] labels shape: (1190,)
[STATUS] training started...
[STATUS] splitted train and test data...
Train data  : (1071, 532)
Test data   : (119, 532)
Train labels: (1071,)
Test labels : (119,)
LR: 0.499602 (0.075640)
LDA: 0.406205 (0.071290)
KNN: 0.358515 (0.046140)
CART: 0.448174 (0.057405)
RF: 0.649913 (0.063204)
NB: 0.385601 (0.052706)
SVM: 0.045786 (0.034080)


<Figure size 640x480 with 1 Axes>

In [1]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
# create the model - Random Forests
clf  = RandomForestClassifier(n_estimators=100, random_state=9)

# fit the training data to the model
clf.fit(trainDataGlobal, trainLabelsGlobal)

# path to test data
test_path = 'dataset/test'
# path to test data
#test_path = "dataset/test"


train_labels = os.listdir(train_path)
train_labels.sort()
print(train_labels)
feature =[]
labels =[]
img_data_list =[]
for training_name in train_labels:
    img_list =os.listdir(train_path+'/'+training_name)
    print('Loaded the images of dataset-' + '{}\n'.format(training_name))
    # get the current training label
    current_label = training_name
    k = 1
# get all the test images paths
#test_images = os.listdir(test_path)
#for x in img_list:
# loop through each image in the test data
#if not admin.isUserAdmin():
#    admin.runAsAdmin()
    for image_path in img_list:
        path = train_path + '/' + training_name + '/'+image_path
        image_path = os.path.join(train_path, training_name, image_path)
        if os.path.exists(image_path):
            # read the image and resize it to a fixed-size
            image = cv2.imread(r'D:\image-classification\dataset\pan.jpg')
            image = cv2.resize(image, fixed_size)
            img_data_list.append(image)

# loop through the test images
#for file in glob.glob(test_path + "/*.jpg"):
    # read the image
 #   image = cv2.imread(file)

    # resize the image
#    image = cv2.resize(image, fixed_size)


    # Global Feature extraction

    fv_hu_moments = fd_hu_moments(image)
    fv_haralick   = fd_haralick(image)
    fv_histogram  = fd_histogram(image)


    # Concatenate global features

    global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])

    # predict label of test image
    prediction = clf.predict(global_feature.reshape(1,-1))[0]

    # show predicted label on image
    final =cv2.putText(image, train_labels[prediction], (20,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,255), 3)

    # display the output image
    final = cv2.cvtColor(final,cv2.COLOR_BGR2RGB)
    plt.imshow(final), plt.show()

    key = cv2.waitKey(0) & 0xFF
    if (key == ord('q')):
        cv2.destroyAllWindows()

NameError: name 'trainDataGlobal' is not defined